## Load modules

In [248]:
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
from pytz import timezone, utc
import numpy as np

import Scripts.config as config
import Scripts.Functions as func
from app import getConfigData
from Scripts.ProcessData_resampler import setUTCDatetime

## Import data

In [249]:
config.io_dir = Path("C:/Users/nms198/OneDrive - Newcastle University/3_Coding/Python/Ti_Rod_BES_BEWISE")
config.update = True

In [250]:
getConfigData()

Importing processed data...
Importing config...


## View data columns

In [251]:
pd.DataFrame({
    'Plot': config.config['dcc_plot_codes'].values(),
    'Traces': config.config['dcc_trace_names'].values()
}, index=config.config['dcc_plot_codes'].keys())

,Plot,Traces
graph0,BES_C,"[BES 1, BES 2, BES 3, BES 4, ∑ [BESs]]"
graph1,BES_CA,"[BES 1A, BES 2A, BES 3A, BES 4A, ∑ [BES A]]"
graph2,BES_CB,"[BES 1B, BES 2B, BES 3B, BES 4B, ∑ [BES B]]"
graph3,BES_CC,"[BES 1C, BES 2C, BES 3C, BES 4C, ∑ [BES C]]"
graph4,BES_V,"[BES 1, BES 2, BES 3, BES 4, ∑ [BESs]]"
graph5,BES_VA,"[BES 1A, BES 2A, BES 3A, BES 4A, ∑ [BES A]]"
graph6,BES_VB,"[BES 1B, BES 2B, BES 3B, BES 4B, ∑ [BES B]]"
graph7,BES_VC,"[BES 1C, BES 2C, BES 3C, BES 4C, ∑ [BES C]]"
graph8,OCP_V,"[BES 1, BES 2, BES 3, BES 4, ∑ [BESs]]"
graph9,OCP_VA,"[BES 1A, BES 2A, BES 3A, BES 4A, ∑ [BES A]]"


## Set variables and create chart data

In [252]:
start = setUTCDatetime('15/02/2022 12:00:00', "UTC")
end = datetime.now(timezone('UTC'))
dates_selected = [func.unixTimeMillis(start), func.unixTimeMillis(end)]
resample = 1
traces = {
    'graph0': ['∑ [BESs]'],
    'graph1': ['∑ [BESs]'],
    'graph2': ['∑ [BESs]'],
    'graph3': ['∑ [BESs]'],
    'graph12': ['Feed sCOD'],
    'graph15': ['Pump Flow Rate']
    }

In [253]:
chart_data = func.create_chart_data(dates_selected, resample, traces)

## View chart data

In [254]:
chart_data

,DateTime,BES_SUM__C_ave,BES_SUM__C_ave_err,sCOD_ave,sCOD_ave_err,PUMP_FR
0,2022-02-15 12:00:00+00:00,0.014258,0.029422,NaN,NaN,-1.2
1,2022-02-15 12:01:00+00:00,-0.019563,0.056319,NaN,NaN,-1.2
2,2022-02-15 12:02:00+00:00,-0.039789,0.034573,NaN,NaN,-1.0
3,2022-02-15 12:03:00+00:00,0.005968,0.040414,NaN,NaN,-1.1
4,2022-02-15 12:04:00+00:00,0.062336,0.045954,NaN,NaN,-1.2
...,...,...,...,...,...,...
24535,2022-03-04 12:55:00+00:00,16.166495,2.557448,NaN,NaN,2.3
24536,2022-03-04 12:56:00+00:00,16.311061,2.514559,NaN,NaN,2.3
24537,2022-03-04 12:57:00+00:00,16.300450,2.571713,NaN,NaN,2.4
24538,2022-03-04 12:58:00+00:00,16.411859,2.563309,NaN,NaN,2.3


## Modify data for calibration

In [255]:
# Path length volumes from wastewater sampling point (start of Marprene tubing) up to:
path_vols = {
    "pre_BES_vol": 85.8,
    #"mid_BES_vol": 189.8,
    #"post_BES_vol": 293.8,

    #"pre_OS_vol": 77.3,
    #"mid_OS_vol": 99.4,
    #"post_OS_vol": 121.6,
}

In [256]:
# Time offset
# Add cumulative volume passed column

chart_data['Vol_passed'] = (
    ((chart_data['DateTime'] - chart_data['DateTime'].shift()).dt.total_seconds()/60) * 
    chart_data['PUMP_FR'].shift()
).cumsum()

In [257]:
chart_data = chart_data.drop(chart_data.query('Vol_passed.isnull()', engine='python').index)
chart_data = chart_data.drop(chart_data.query('BES_SUM__C_ave.isnull()', engine='python').index)
chart_data = chart_data.reset_index(drop=True)
chart_data

,DateTime,BES_SUM__C_ave,BES_SUM__C_ave_err,sCOD_ave,sCOD_ave_err,PUMP_FR,Vol_passed
0,2022-02-15 12:01:00+00:00,-0.019563,0.056319,NaN,NaN,-1.2,-1.2
1,2022-02-15 12:02:00+00:00,-0.039789,0.034573,NaN,NaN,-1.0,-2.4
2,2022-02-15 12:03:00+00:00,0.005968,0.040414,NaN,NaN,-1.1,-3.4
3,2022-02-15 12:04:00+00:00,0.062336,0.045954,NaN,NaN,-1.2,-4.5
4,2022-02-15 12:05:00+00:00,0.033157,0.057208,NaN,NaN,-0.8,-5.7
...,...,...,...,...,...,...,...
23394,2022-03-04 12:55:00+00:00,16.166495,2.557448,NaN,NaN,2.3,104074.8
23395,2022-03-04 12:56:00+00:00,16.311061,2.514559,NaN,NaN,2.3,104077.1
23396,2022-03-04 12:57:00+00:00,16.300450,2.571713,NaN,NaN,2.4,104079.4
23397,2022-03-04 12:58:00+00:00,16.411859,2.563309,NaN,NaN,2.3,104081.8


In [258]:
chart_data['Vol_passed_shifted'] = chart_data['Vol_passed'].shift(-1)
for path in path_vols.keys():
    chart_data[path] = chart_data['Vol_passed'] + path_vols[path]
    chart_data[path[0:-4]] = np.nan
chart_data

,DateTime,BES_SUM__C_ave,BES_SUM__C_ave_err,sCOD_ave,sCOD_ave_err,PUMP_FR,Vol_passed,Vol_passed_shifted,pre_BES_vol,pre_BES
0,2022-02-15 12:01:00+00:00,-0.019563,0.056319,NaN,NaN,-1.2,-1.2,-2.4,84.6,NaN
1,2022-02-15 12:02:00+00:00,-0.039789,0.034573,NaN,NaN,-1.0,-2.4,-3.4,83.4,NaN
2,2022-02-15 12:03:00+00:00,0.005968,0.040414,NaN,NaN,-1.1,-3.4,-4.5,82.4,NaN
3,2022-02-15 12:04:00+00:00,0.062336,0.045954,NaN,NaN,-1.2,-4.5,-5.7,81.3,NaN
4,2022-02-15 12:05:00+00:00,0.033157,0.057208,NaN,NaN,-0.8,-5.7,-6.5,80.1,NaN
...,...,...,...,...,...,...,...,...,...,...
23394,2022-03-04 12:55:00+00:00,16.166495,2.557448,NaN,NaN,2.3,104074.8,104077.1,104160.6,NaN
23395,2022-03-04 12:56:00+00:00,16.311061,2.514559,NaN,NaN,2.3,104077.1,104079.4,104162.9,NaN
23396,2022-03-04 12:57:00+00:00,16.300450,2.571713,NaN,NaN,2.4,104079.4,104081.8,104165.2,NaN
23397,2022-03-04 12:58:00+00:00,16.411859,2.563309,NaN,NaN,2.3,104081.8,104084.1,104167.6,NaN


In [259]:
vals = chart_data.loc[chart_data.query("Vol_passed_shifted > pre_BES_vol").index + 1, 'BES_SUM__C_ave']
vals.index -= 1
chart_data.loc[chart_data.query("Vol_passed_shifted > pre_BES_vol").index, path[0:-4]] = vals

In [260]:
chart_data.query("Vol_passed_shifted > pre_BES_vol")

,DateTime,BES_SUM__C_ave,BES_SUM__C_ave_err,sCOD_ave,sCOD_ave_err,PUMP_FR,Vol_passed,Vol_passed_shifted,pre_BES_vol,pre_BES
1038,2022-02-16 06:13:00+00:00,23.286358,2.091558,NaN,NaN,379.4,7367.5,7746.9,7453.3,23.647109
1039,2022-02-16 06:14:00+00:00,23.647109,2.132515,NaN,NaN,379.5,7746.9,8126.4,7832.7,23.726023
1040,2022-02-16 06:15:00+00:00,23.726023,2.117202,NaN,NaN,379.5,8126.4,8505.9,8212.2,23.789022
1730,2022-02-16 17:45:00+00:00,28.779524,2.306867,NaN,NaN,2.3,10131.6,12051.3,10217.4,36.340379
2448,2022-02-17 06:13:00+00:00,29.930413,1.508562,NaN,NaN,379.4,14454.6,14834.0,14540.4,30.355490
...,...,...,...,...,...,...,...,...,...,...
22330,2022-03-03 17:44:00+00:00,28.631311,0.386228,NaN,NaN,2.4,99361.2,100417.8,99447.0,40.122630
23048,2022-03-04 06:12:00+00:00,22.928591,2.360285,NaN,NaN,98.8,102841.6,102940.4,102927.4,22.847687
23049,2022-03-04 06:13:00+00:00,22.847687,2.423536,NaN,NaN,98.8,102940.4,103039.2,103026.2,22.663332
23050,2022-03-04 06:14:00+00:00,22.663332,2.470340,NaN,NaN,98.9,103039.2,103138.1,103125.0,22.416642
